In [6]:
# ETL Data Processor for Google Colab

#Tapi Goredema and Edward Cho-Jung

import pandas as pd
import json
import sqlite3
import os



!pip install kagglehub --quiet
import kagglehub

# Download dataset from Kaggle
try:
    path = kagglehub.dataset_download("nathanlauga/nba-games")
    remote_df = pd.read_csv(os.path.join(path, "games_details.csv"))
    print("Remote dataset downloaded and loaded successfully.")
except Exception as e:
    print(f"Error downloading remote dataset: {e}")

# 2. Load local data
local_csv_path = "/content/games.csv"  # Assuming you uploaded the file to Colab session

try:
    local_df = pd.read_csv(local_csv_path)
    print("Local dataset loaded successfully.")
except Exception as e:
    print(f"Error loading local dataset: {e}")

#3. Data transformation

def convert_data_format(df, output_format, output_path):
    """Converts a DataFrame to the specified format and saves it to the given path."""
    try:
        if output_format == "csv":
            df.to_csv(output_path, index=False)
        elif output_format == "json":
            df.to_json(output_path, orient="records", lines=True)
        elif output_format == "sql":
            conn = sqlite3.connect(output_path)
            df.to_sql("transformed_data", conn, if_exists="replace", index=False)
            conn.close()
        print(f"Data successfully saved as {output_format} at {output_path}")
    except Exception as e:
        print(f"Error converting data to {output_format}: {e}")

output_format = input("Enter output format (csv, json, sql, none): ").strip().lower()
if output_format in ["csv", "json", "sql"]:
    local_output_path = f"/content/local_transformed_data.{output_format}"
    remote_output_path = f"/content/remote_transformed_data.{output_format}"
    convert_data_format(local_df, output_format, local_output_path)
    convert_data_format(remote_df, output_format, remote_output_path)
else:
    print("No data conversion selected.")

# 4. removing columns
column_to_remove_local = "column_name_to_remove_from_local"
column_to_remove_remote = "column_name_to_remove_from_remote"

if column_to_remove_local in local_df.columns:
    local_df.drop(columns=[column_to_remove_local], inplace=True)
    print(f"Column '{column_to_remove_local}' removed from local dataset.")

if column_to_remove_remote in remote_df.columns:
    remote_df.drop(columns=[column_to_remove_remote], inplace=True)
    print(f"Column '{column_to_remove_remote}' removed from remote dataset.")

# Merge datasets after transformation
merged_transformed = pd.concat([remote_df, local_df], axis=0, ignore_index=True, sort=False)

#5. Data analysis and summary
try:
    # Summary of ingested data
    print("\nIngested (Raw) Data Summary:")
    print(f"Local CSV - Records: {len(local_df)}, Columns: {len(local_df.columns) + 1}")
    print(f"Remote CSV - Records: {len(remote_df)}, Columns: {len(remote_df.columns) + 1}")

    # Summary of transformed data before merging
    print("\nTransformed Data Summary (After Column Removal, Before Merge):")
    print(f"Transformed Local Data - Records: {len(local_df)}, Columns: {len(local_df.columns)}")
    print(f"Transformed Remote Data - Records: {len(remote_df)}, Columns: {len(remote_df.columns)}")

    # Summary of transformed merged data
    print("\nTransformed Merged Data Summary:")
    print(f"Records: {len(merged_transformed)}, Columns: {len(merged_transformed.columns)}")

    # Correlation matrix for the transformed merged dataset
    print("\nCorrelation Matrix for Transformed Merged Data:")
    print(merged_transformed.corr(numeric_only=True))

    # Basic aggregations for transformed merged data
    print("\nAggregated Summary Statistics for Transformed Merged Data:")
    print(merged_transformed.describe())

except Exception as e:
    print(f"Error summarizing datasets: {e}")

# 6. sql database
db_path = f"/content/etl_output.db"
try:
    conn = sqlite3.connect(db_path)
    merged_transformed.to_sql("merged_transformed_data", conn, if_exists="replace", index=False)
    print("Transformed merged data stored in SQLite database.")
except Exception as e:
    print(f"Error saving merged transformed data to SQL database: {e}")
finally:
    conn.close()

# Query SQLite Database
try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in database:", tables)

    df = pd.read_sql("SELECT * FROM merged_transformed_data LIMIT 5;", conn)
    print(df)
except Exception as e:
    print(f"Error querying database: {e}")
finally:
    conn.close()


<ipython-input-6-d90e40af74b3>:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  remote_df = pd.read_csv(os.path.join(path, "games_details.csv"))


Remote dataset downloaded and loaded successfully.
Local dataset loaded successfully.
Enter output format (csv, json, sql, none): sql
Data successfully saved as sql at /content/local_transformed_data.sql
Data successfully saved as sql at /content/remote_transformed_data.sql

Ingested (Raw) Data Summary:
Local CSV - Records: 26651, Columns: 22
Remote CSV - Records: 668628, Columns: 30

Transformed Data Summary (After Column Removal, Before Merge):
Transformed Local Data - Records: 26651, Columns: 21
Transformed Remote Data - Records: 668628, Columns: 29

Transformed Merged Data Summary:
Records: 695279, Columns: 49

Correlation Matrix for Transformed Merged Data:
                  GAME_ID   TEAM_ID  PLAYER_ID       FGM       FGA    FG_PCT  \
GAME_ID          1.000000 -0.034193  -0.004770  0.032073  0.035696 -0.011566   
TEAM_ID         -0.034193  1.000000   0.005020 -0.004936 -0.002352 -0.000025   
PLAYER_ID       -0.004770  0.005020   1.000000 -0.005698 -0.006998 -0.002473   
FGM      